### 연관성 분석 - FP Growth.

In [ ]:
# 다음 모듈을 설치한다.
# !pip install mlxtend

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from mlxtend.preprocessing import TransactionEncoder
from mlxtend.frequent_patterns import apriori, fpgrowth, association_rules

#### 1. 데이터를 읽어와서 탐색한다.

In [ ]:
df = pd.read_csv("../data/data_basket.csv")

In [ ]:
df.shape

In [ ]:
df.info()

In [ ]:
df.head()

In [ ]:
# 장바구니 크기의 도수표를 만들고 시각화로 출력한다.
size_frequencies = df['items'].apply(lambda x: x.count(',') +1).value_counts()
size_frequencies.plot.bar(color = 'orange', alpha=0.7)
plt.xlabel("Basket Size")
plt.show()

#### 2. 데이터 전처리.

**2.1. 전처리 준비.**

In [ ]:
# 문자열에서 중괄호를 제거한다.
sr = df['items'].apply(lambda x: x[1:-1])

In [ ]:
# List of List 로 변환한다.
my_data = []
for x in sr:
    a_basket = x.split(",")
    if len(a_basket) > 1:                           # 단 한개의 아이템만 있는 장바구니는 필요 없다.
        my_data += [a_basket]

In [ ]:
# 장바구니의 갯수.
print(len(my_data))

In [ ]:
# 첫 3개의 장바구니를 출력해 본다.
my_data[:3]

**2.2. Encoding된 형태로 변환.**

In [ ]:
# TransactionEncoder 객체 생성.
my_te = TransactionEncoder()
my_data_array = my_te.fit_transform(my_data)       
type(my_data_array)                              # Numpy 배열 객체 확인.

In [ ]:
# Numpy 배열의 shape.
# 출력하기에는 크다.
my_data_array.shape

In [ ]:
# 학습된 columns_의 일부를 출력해 본다.
print(len(my_te.columns_))       # columns_는 list 이다. 이것의 길이를 출력해 본다.
print(my_te.columns_[:10])

In [ ]:
# Numpy 배열을 역으로 변환할 수도 있다.
my_te.inverse_transform(my_data_array)[:3]

In [ ]:
# 이제는 데이터 프레임으로 변환한다.
df_processed = pd.DataFrame(data=my_data_array, columns = my_te.columns_)
print(df_processed.shape)            # Shape 출력.        
df_processed.head(5)                 # 상단 출력.

#### 3. FP Growth 알고리즘 적용.

In [ ]:
# 지지도(support)가 최소값 이상인 항목들만 걸러낸다.
my_frequent_patterns = fpgrowth(df_processed, min_support=0.001, use_colnames=True)

In [ ]:
# 항목의 지지도(support)가 높은 순서대로 출력해 본다.
# 항목의 지지도(support)는 항목이 포함된 basket의 비율을 의미한다.
my_frequent_patterns.sort_values(by="support", ascending=False).head(100)

In [ ]:
# 지지도(support)가 높은 항목들을 기반으로 연관 규칙을 만들어 본다.
# 신뢰도(confidence)가 최소값 이상인 규칙들만 걸러낸다.
my_rules = association_rules(my_frequent_patterns, metric="confidence",min_threshold=0.75)

In [ ]:
# 향상도(lift)가 1 이상인 연관 규칙들만 걸러낸다.
my_rules = my_rules[my_rules.lift > 1.0]
# 향상도(lift)가 높은 순서대로 연관 규칙을 출력해 본다.
my_rules[['antecedents', 'consequents','support','confidence','lift']].sort_values(by='lift',ascending=False ).head(10)

In [ ]:
# 왼편, 오른편의 항목수를 명시해서 연관 규칙을 출력해 본다.
n_left = 3
n_right = 1
my_rules_small = my_rules[(my_rules.antecedents.apply(lambda x: len(x)) == n_left) & (my_rules.consequents.apply(lambda x: len(x)) == n_right)]
my_rules_small[['antecedents', 'consequents','support','confidence','lift']].sort_values(by='lift',ascending=False ).head(10)